# Schools

In [1]:
import geopandas as gpd
import folium

In [2]:
print(gpd.list_layers('data/apsschools.zip'))

             name geometry_type
0  schoolLocation         Point
1            high       Polygon
2          middle       Polygon
3            elem       Polygon


In [3]:
school_locations = gpd.read_file('data/apsschools.zip', layer='schoolLocation').to_crs(4267)
school_locations.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   OBJECTID    198 non-null    int64         
 1   NAME        198 non-null    object        
 2   TYPE        198 non-null    object        
 3   GRADELEVEL  198 non-null    object        
 4   AUTHORIZER  198 non-null    object        
 5   SCHOOLOFCH  5 non-null      object        
 6   LASTUPDATE  198 non-null    datetime64[ms]
 7   created_us  18 non-null     object        
 8   created_da  18 non-null     datetime64[ms]
 9   last_edite  22 non-null     object        
 10  last_edi_1  22 non-null     datetime64[ms]
 11  geometry    198 non-null    geometry      
dtypes: datetime64[ms](3), geometry(1), int64(1), object(7)
memory usage: 18.7+ KB


In [4]:
school_locations.drop(columns=['LASTUPDATE', 'created_da', 'created_us', 
                              'created_da', 'last_edite', 'last_edi_1'], 
                     inplace=True)
school_locations.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   OBJECTID    198 non-null    int64   
 1   NAME        198 non-null    object  
 2   TYPE        198 non-null    object  
 3   GRADELEVEL  198 non-null    object  
 4   AUTHORIZER  198 non-null    object  
 5   SCHOOLOFCH  5 non-null      object  
 6   geometry    198 non-null    geometry
dtypes: geometry(1), int64(1), object(5)
memory usage: 11.0+ KB


In [5]:
school_locations.iloc[0]

OBJECTID                                                  1
NAME                              The Montessori Elementary
TYPE                                                Charter
GRADELEVEL                                Elementary/Middle
AUTHORIZER           New Mexico Public Education Commission
SCHOOLOFCH                                             None
geometry      POINT (1517775.0707261264 1506701.7353094593)
Name: 0, dtype: object

In [6]:
school_locations[school_locations.SCHOOLOFCH.notnull()]

,OBJECTID,NAME,TYPE,GRADELEVEL,AUTHORIZER,SCHOOLOFCH,geometry
63,73,John Adams,Traditional,Middle,Albuquerque Public School District,Vision Quest,POINT (1505528.409 1491575.165)
98,109,Wilson,Traditional,Middle,Albuquerque Public School District,Transitions Outcomes Programs,POINT (1542062.945 1477944.355)
121,132,Hayes,Traditional,Middle,Albuquerque Public School District,Vision Quest,POINT (1549455.856 1488247.15)
134,145,Highland,Traditional,High,Albuquerque Public School District,APS Autism Center,POINT (1538349.581 1482729.427)
172,184,Harrison,Traditional,Middle,Albuquerque Public School District,Vision Quest,POINT (1512457.452 1462025.907)


In [7]:
school_locations[school_locations.NAME == 'John Adams']

,OBJECTID,NAME,TYPE,GRADELEVEL,AUTHORIZER,SCHOOLOFCH,geometry
63,73,John Adams,Traditional,Middle,Albuquerque Public School District,Vision Quest,POINT (1505528.409 1491575.165)


In [8]:
school_locations.drop(columns=['SCHOOLOFCH'], inplace=True)
school_locations.rename(columns={
    'NAME':'Name', 
    'TYPE':'Type', 
    'GRADELEVEL':'GradeLevel', 
    'AUTHORIZER': 'Authority'
}, inplace=True)
school_locations.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   OBJECTID    198 non-null    int64   
 1   Name        198 non-null    object  
 2   Type        198 non-null    object  
 3   GradeLevel  198 non-null    object  
 4   Authority   198 non-null    object  
 5   geometry    198 non-null    geometry
dtypes: geometry(1), int64(1), object(4)
memory usage: 9.4+ KB


In [10]:
school_locations = school_locations.to_crs(4267)
school_locations.crs

<Geographic 2D CRS: EPSG:4267>
Name: NAD27
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North and central America: Antigua and Barbuda - onshore. Bahamas - onshore plus offshore over internal continental shelf only. Belize - onshore. British Virgin Islands - onshore. Canada onshore - Alberta, British Columbia, Manitoba, New Brunswick, Newfoundland and Labrador, Northwest Territories, Nova Scotia, Nunavut, Ontario, Prince Edward Island, Quebec, Saskatchewan and Yukon - plus offshore east coast. Cuba - onshore and offshore. El Salvador - onshore. Guatemala - onshore. Honduras - onshore. Panama - onshore. Puerto Rico - onshore. Mexico - onshore plus offshore east coast. Nicaragua - onshore. United States (USA) onshore and offshore - Alabama, Alaska, Arizona, Arkansas, California, Colorado, Connecticut, Delaware, Florida, Georgia, Idaho, Illinois, Indiana, Iowa, Kansas, Kentucky, Louisiana, Maine, Maryland, 

In [12]:
def style_function(feature): 
    return {
        'color': 'transparent', 
        'fillColor': 'transparent'
    }

tooltip = folium.features.GeoJsonTooltip(
    fields=('Name', 'Type', 'GradeLevel','Authority'), 
    aliases=("Name:", "Type:", "Grade Level:", "Authority:")
)

interactive_map = folium.Map(
    location=(35.08, -106.65), 
    zoom_start=11.3, 
    control_scale=True, 
    tiles="https://mt1.google.com/vt/lyrs=r&x={x}&y={y}&z={z}",
    attr="Google maps",
)
addresses_layer = folium.features.GeoJson(
    school_locations, 
    name='School Locations'
)
addresses_layer.add_to(interactive_map)

tooltip_layer = folium.features.GeoJson(
    school_locations, 
    style_function=style_function, 
    tooltip=tooltip
)
tooltip_layer.add_to(interactive_map)

interactive_map

In [74]:
high_schools = gpd.read_file('data/apsschools.zip', layer='high').to_crs(4267)
high_schools.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   OBJECTID    13 non-null     int64         
 1   SCHOOLSERV  13 non-null     object        
 2   SCHOOLADDR  13 non-null     object        
 3   SCHOOLPHON  13 non-null     object        
 4   created_us  13 non-null     object        
 5   created_da  13 non-null     datetime64[ms]
 6   last_edite  13 non-null     object        
 7   last_edi_1  13 non-null     datetime64[ms]
 8   STAr        13 non-null     float64       
 9   STLe        13 non-null     float64       
 10  geometry    13 non-null     geometry      
dtypes: datetime64[ms](2), float64(2), geometry(1), int64(1), object(5)
memory usage: 1.2+ KB


In [75]:
high_schools.drop(columns=['created_us', 'created_da', 'last_edite', 
                         'last_edi_1', 'STAr', 'STLe'], 
                inplace=True)

In [76]:
high_schools.rename(columns=dict(
    SCHOOLSERV = 'Name', 
    SCHOOLADDR = 'Address', 
    SCHOOLPHON = 'Phone'
), inplace=True)

In [77]:
high_schools['type'] = 'High'
high_schools.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   OBJECTID  13 non-null     int64   
 1   Name      13 non-null     object  
 2   Address   13 non-null     object  
 3   Phone     13 non-null     object  
 4   geometry  13 non-null     geometry
 5   type      13 non-null     object  
dtypes: geometry(1), int64(1), object(4)
memory usage: 756.0+ bytes


In [78]:
middle_schools = gpd.read_file('data/apsschools.zip', layer='middle').to_crs(4267)
middle_schools.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   OBJECTID    29 non-null     int64         
 1   SCHOOLSERV  29 non-null     object        
 2   SCHOOLADDR  29 non-null     object        
 3   SCHOOLPHON  28 non-null     object        
 4   created_us  29 non-null     object        
 5   created_da  29 non-null     datetime64[ms]
 6   last_edite  29 non-null     object        
 7   last_edi_1  29 non-null     datetime64[ms]
 8   STAr        29 non-null     float64       
 9   STLe        29 non-null     float64       
 10  geometry    29 non-null     geometry      
dtypes: datetime64[ms](2), float64(2), geometry(1), int64(1), object(5)
memory usage: 2.6+ KB


In [79]:
middle_schools.rename(columns=dict(
    SCHOOLSERV = 'Name', 
    SCHOOLADDR = 'Address', 
    SCHOOLPHON = 'Phone'
), inplace=True)
middle_schools.drop(columns=['created_us', 'created_da', 'last_edite', 
                         'last_edi_1', 'STAr', 'STLe'], 
                    inplace=True)
middle_schools['type'] = 'Middle'
middle_schools.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   OBJECTID  29 non-null     int64   
 1   Name      29 non-null     object  
 2   Address   29 non-null     object  
 3   Phone     28 non-null     object  
 4   geometry  29 non-null     geometry
 5   type      29 non-null     object  
dtypes: geometry(1), int64(1), object(4)
memory usage: 1.5+ KB


In [80]:
elementary_schools = gpd.read_file('data/apsschools.zip', layer='elem').to_crs(4267)
elementary_schools.rename(columns=dict(
    SCHOOLSERV = 'Name', 
    SCHOOLADDR = 'Address', 
    SCHOOLPHON = 'Phone'
), inplace=True)
elementary_schools.drop(columns=['created_us', 'created_da', 'last_edite', 
                         'last_edi_1', 'STAr', 'STLe'], 
                    inplace=True)
elementary_schools['type'] = 'Elementary'
elementary_schools.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   OBJECTID  89 non-null     int64   
 1   Name      89 non-null     object  
 2   Address   89 non-null     object  
 3   Phone     86 non-null     object  
 4   geometry  89 non-null     geometry
 5   type      89 non-null     object  
dtypes: geometry(1), int64(1), object(4)
memory usage: 4.3+ KB


In [81]:
import pandas as pd

In [82]:
schools = pd.concat([high_schools, middle_schools, elementary_schools])

In [83]:
schools.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 131 entries, 0 to 88
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   OBJECTID  131 non-null    int64   
 1   Name      131 non-null    object  
 2   Address   131 non-null    object  
 3   Phone     127 non-null    object  
 4   geometry  131 non-null    geometry
 5   type      131 non-null    object  
dtypes: geometry(1), int64(1), object(4)
memory usage: 7.2+ KB


In [86]:
schools.to_file('../../../data/local/abq_city_data.gpkg', layer='school_service_areas')

In [87]:
school_locations.to_file('../../../data/local/abq_city_data.gpkg', layer='schools')

In [88]:
schools.to_file('../../../localData/City/schools.gpkg', 
               layer='coverage_areas')

In [89]:
school_locations.to_file('../../../localData/City/schools.gpkg', 
                        layer='locations')